In [15]:
# ============================================================================
# DATA MANAGER DASHBOARD - VERSIÓN GOOGLE COLAB
# Sistema completo de gestión, análisis y predicción de datos
# ============================================================================

# INSTALACIONES
!pip install -q xlsxwriter openpyxl scikit-learn ipywidgets

# IMPORTS
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import io
import warnings
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')

# ============================================================================
# VARIABLES GLOBALES
# ============================================================================
dataframes = {}
historial_cambios = []

# ============================================================================
# FUNCIONES DE GESTIÓN DE DATOS
# ============================================================================

def registrar_cambio(accion, detalles):
    """Registra cambios en el historial"""
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    historial_cambios.append({
        'timestamp': timestamp,
        'accion': accion,
        'detalles': detalles
    })

def actualizar_lista_archivos():
    """Actualiza la visualización de archivos cargados"""
    if not dataframes:
        lista_archivos.value = '<p style="color: gray;">📂 No hay archivos cargados</p>'
        combo_csv.options = []
        combo_modificar.options = []
        combo_eliminar.options = []
        combo_analisis.options = []
        return

    html_lista = '<div style="background-color: #f8f9fa; padding: 15px; border-radius: 5px;">'
    html_lista += f'<h4 style="color: #007bff;">📁 Archivos Cargados ({len(dataframes)})</h4>'

    for idx, (name, df) in enumerate(dataframes.items(), 1):
        try:
            num_cols = len(df.select_dtypes(include='number').columns)
            cat_cols = len(df.select_dtypes(include='object').columns)

            html_lista += f'''
            <div style="background-color: white; padding: 10px; margin: 5px 0; border-radius: 5px; border-left: 4px solid #28a745;">
                <strong>{idx}. {name}</strong><br>
                <small style="color: #6c757d;">
                    📏 {df.shape[0]} filas × {df.shape[1]} columnas |
                    🔢 {num_cols} numéricas |
                    📝 {cat_cols} categóricas
                </small>
            </div>
            '''
        except Exception as e:
            html_lista += f'<div style="color: red;">Error: {name}</div>'

    html_lista += '</div>'
    lista_archivos.value = html_lista

    # Actualizar todos los combos
    opciones = list(dataframes.keys())
    combo_csv.options = opciones
    combo_modificar.options = opciones
    combo_eliminar.options = opciones
    combo_analisis.options = opciones
    combo_merge.options = opciones

# ============================================================================
# FUNCIONES DE ANÁLISIS Y REPORTES
# ============================================================================

def ranking():
    """Ranking de clientes por total"""
    try:
        if 'clientes.csv' not in dataframes or 'facturas_enc.csv' not in dataframes:
            return None

        clientes = dataframes['clientes.csv']
        facturas = dataframes['facturas_enc.csv']

        df = pd.merge(facturas, clientes, on='id_cliente', how='left')
        ranking = df.groupby('nombre')['total'].sum().reset_index()
        ranking = ranking.sort_values(by='total', ascending=False)

        return ranking
    except Exception as e:
        print(f"Error: {e}")
        return None

def ticket_promedio():
    """Calcula ticket promedio por cliente"""
    try:
        if 'clientes.csv' not in dataframes or 'facturas_enc.csv' not in dataframes:
            return None

        clientes = dataframes['clientes.csv']
        facturas = dataframes['facturas_enc.csv']

        df = pd.merge(facturas, clientes, on='id_cliente', how='left')
        ticket = df.groupby('nombre')['total'].mean().reset_index()
        ticket.rename(columns={'total': 'ticket_promedio'}, inplace=True)
        ticket = ticket.sort_values(by='ticket_promedio', ascending=False)

        return ticket
    except Exception as e:
        print(f"Error: {e}")
        return None

def ventas_por_mes():
    """Ventas agrupadas por mes"""
    try:
        if 'ventas.csv' not in dataframes:
            return None

        ventas = dataframes['ventas.csv']

        if 'fecha' not in ventas.columns:
            return None

        ventas['fecha'] = pd.to_datetime(ventas['fecha'], errors='coerce')
        ventas_mes = ventas.groupby(ventas['fecha'].dt.to_period('M'))['precio_venta'].sum().reset_index()

        return ventas_mes
    except Exception as e:
        print(f"Error: {e}")
        return None

def top_productos():
    """Productos más vendidos"""
    try:
        if 'productos.csv' not in dataframes or 'facturas_det.csv' not in dataframes:
            return None

        productos = dataframes['productos.csv']
        facturas = dataframes['facturas_det.csv']

        df = pd.merge(facturas, productos, on='id_producto', how='left')
        top = df.groupby('descripcion')['cantidad'].sum().reset_index()
        top = top.sort_values(by='cantidad', ascending=False)

        return top
    except Exception as e:
        print(f"Error: {e}")
        return None

# ============================================================================
# EVENTOS DE INTERFAZ
# ============================================================================

def on_upload_change(change):
    """Maneja carga de archivos - ACUMULA"""
    with output_carga:
        clear_output()

        if not uploader.value:
            print("⚠️ No hay archivos seleccionados")
            return

        print("📂 Procesando archivos...")

        archivos_nuevos = 0
        for name, file_info in uploader.value.items():
            if name not in dataframes:
                try:
                    df = pd.read_csv(io.BytesIO(file_info['content']))
                    dataframes[name] = df
                    archivos_nuevos += 1
                    registrar_cambio("CSV Cargado", f"{name} - {df.shape[0]} filas")
                    print(f"✅ {name}: {df.shape[0]} filas × {df.shape[1]} columnas")
                except Exception as e:
                    print(f"❌ Error con {name}: {e}")

        if archivos_nuevos > 0:
            print(f"\n✅ {archivos_nuevos} archivo(s) nuevo(s)")
            print(f"📊 Total: {len(dataframes)} archivo(s)")
        else:
            print("ℹ️ No hay archivos nuevos")

        actualizar_lista_archivos()

def ver_datos_completos(button):
    """Muestra todos los datos del CSV seleccionado"""
    with output_visualizar:
        clear_output()

        nombre = combo_csv.value

        if not nombre or nombre not in dataframes:
            print("⚠️ Selecciona un archivo")
            return

        df = dataframes[nombre]

        print(f"📊 {nombre}")
        print("="*70)
        print(f"📏 Dimensiones: {df.shape[0]} filas × {df.shape[1]} columnas")
        print(f"🔢 Numéricas: {len(df.select_dtypes(include='number').columns)}")
        print(f"📝 Categóricas: {len(df.select_dtypes(include='object').columns)}")
        print(f"⚠️ Nulos: {df.isnull().sum().sum()}")
        print(f"🔄 Duplicados: {df.duplicated().sum()}")
        print("="*70)
        print("\n📋 DATOS COMPLETOS (SIN OMITIR REGISTROS):")
        display(df)

        print(f"\n✅ Mostrando todas las {len(df)} filas")

def anadir_fila(button):
    """Añade nueva fila con ID autoincremental"""
    with output_modificar:
        clear_output()

        nombre = combo_modificar.value

        if not nombre or nombre not in dataframes:
            print("⚠️ Selecciona un archivo")
            return

        df = dataframes[nombre]

        print(f"➕ Añadiendo fila a: {nombre}")
        print("="*70)
        print(f"📋 Columnas: {', '.join(df.columns)}")
        print("\n⚠️ Para IDs: se generarán automáticamente")
        print("Ingresa valores para cada columna:\n")

        nueva_fila = {}
        for col in df.columns:
            if col.lower() in ['id', 'id_cliente', 'id_producto', 'id_factura']:
                nuevo_id = len(df) + 1
                nueva_fila[col] = nuevo_id
                print(f"🔑 {col}: {nuevo_id} (autoincremental)")
            else:
                valor = input(f"📝 {col}: ")
                nueva_fila[col] = valor

        # Añadir fila
        df_nuevo = pd.concat([df, pd.DataFrame([nueva_fila])], ignore_index=True)
        dataframes[nombre] = df_nuevo

        registrar_cambio("Fila Añadida", nombre)

        print("\n✅ Fila añadida correctamente")
        print(f"📊 Nuevas dimensiones: {df_nuevo.shape[0]} filas")

def eliminar_fila(button):
    """Elimina fila por índice"""
    with output_eliminar:
        clear_output()

        nombre = combo_eliminar.value

        if not nombre or nombre not in dataframes:
            print("⚠️ Selecciona un archivo")
            return

        df = dataframes[nombre]

        print(f"🗑️ Eliminar fila de: {nombre}")
        print("="*70)
        display(df.head(10))

        try:
            fila = int(input(f"\n📍 Índice de fila a eliminar (0-{len(df)-1}): "))

            if 0 <= fila < len(df):
                df_nuevo = df.drop(fila).reset_index(drop=True)
                dataframes[nombre] = df_nuevo

                registrar_cambio("Fila Eliminada", f"{nombre} - fila {fila}")

                print(f"\n✅ Fila {fila} eliminada")
                print(f"📊 Nuevas dimensiones: {df_nuevo.shape[0]} filas")
            else:
                print("❌ Índice inválido")
        except ValueError:
            print("❌ Debes ingresar un número")

def modificar_celda(button):
    """Modifica una celda específica"""
    with output_modificar:
        clear_output()

        nombre = combo_modificar.value

        if not nombre or nombre not in dataframes:
            print("⚠️ Selecciona un archivo")
            return

        df = dataframes[nombre]

        print(f"✏️ Modificar celda en: {nombre}")
        print("="*70)
        print(f"📋 Columnas: {', '.join(df.columns)}")

        try:
            fila = int(input(f"\n📍 Fila (0-{len(df)-1}): "))
            columna = input("📋 Columna: ").strip()

            if fila < 0 or fila >= len(df):
                print("❌ Fila inválida")
                return

            if columna not in df.columns:
                print("❌ Columna inválida")
                return

            print(f"\n📝 Valor actual: {df.at[fila, columna]}")
            nuevo_valor = input("✨ Nuevo valor: ")

            df.at[fila, columna] = nuevo_valor
            dataframes[nombre] = df

            registrar_cambio("Celda Modificada", f"{nombre} - fila {fila}, col {columna}")

            print("\n✅ Celda modificada correctamente")
        except ValueError:
            print("❌ Error en los valores ingresados")

def unificar_tablas(button):
    """Une múltiples tablas con MERGE"""
    with output_merge:
        clear_output()

        if len(dataframes) < 2:
            print("⚠️ Se necesitan al menos 2 archivos")
            return

        print("🔗 UNIFICAR TABLAS")
        print("="*70)
        print("Archivos disponibles:")
        for idx, nombre in enumerate(dataframes.keys(), 1):
            print(f"{idx}. {nombre}")

        try:
            indices = input("\nÍndices a unir (ej: 1,2,3): ")
            indices = [int(x.strip())-1 for x in indices.split(",") if x.strip().isdigit()]

            if len(indices) < 2:
                print("❌ Debes seleccionar al menos 2")
                return

            nombres = list(dataframes.keys())
            df_merge = dataframes[nombres[indices[0]]]
            nombre_resultado = nombres[indices[0]]

            for idx in indices[1:]:
                nombre = nombres[idx]
                df = dataframes[nombre]

                print(f"\nUniendo {nombre_resultado} con {nombre}")
                print(f"Columnas en {nombre_resultado}: {', '.join(df_merge.columns)}")
                print(f"Columnas en {nombre}: {', '.join(df.columns)}")

                columna = input("🔑 Columna común: ").strip()
                tipo = input("Tipo de JOIN (left/right/inner/outer): ").strip().lower()

                if tipo not in ['left', 'right', 'inner', 'outer']:
                    tipo = 'left'

                df_merge = pd.merge(df_merge, df, on=columna, how=tipo)
                nombre_resultado += f"_{nombre}"

            print("\n✅ Tablas unificadas")
            print(f"📊 Dimensiones: {df_merge.shape[0]} filas × {df_merge.shape[1]} columnas")
            display(df_merge.head(10))

            guardar = input("\n💾 ¿Guardar resultado? (s/n): ").strip().lower()
            if guardar == 's':
                dataframes[f"{nombre_resultado}_merged"] = df_merge
                registrar_cambio("Tablas Unificadas", nombre_resultado)
                actualizar_lista_archivos()
                print(f"✅ Guardado como: {nombre_resultado}_merged")

        except Exception as e:
            print(f"❌ Error: {e}")

def ejecutar_reporte(button):
    """Ejecuta reportes predefinidos"""
    with output_reportes:
        clear_output()

        reporte = combo_reportes.value

        print(f"📊 Ejecutando: {reporte}")
        print("="*70)

        resultado = None

        if reporte == "🏆 Ranking de Clientes":
            resultado = ranking()
        elif reporte == "💰 Ticket Promedio":
            resultado = ticket_promedio()
        elif reporte == "📅 Ventas por Mes":
            resultado = ventas_por_mes()
        elif reporte == "🎯 Top Productos":
            resultado = top_productos()

        if resultado is not None:
            print("✅ Reporte generado\n")
            display(resultado.head(20))

            # Gráfico si es posible
            if len(resultado) > 0 and len(resultado.columns) == 2:
                fig, ax = plt.subplots(figsize=(12, 6))
                ax.bar(range(min(10, len(resultado))), resultado.iloc[:10, 1].values, color='steelblue')
                ax.set_xticks(range(min(10, len(resultado))))
                ax.set_xticklabels(resultado.iloc[:10, 0].values, rotation=45, ha='right')
                ax.set_title(reporte, fontsize=14, fontweight='bold')
                plt.tight_layout()
                plt.show()
        else:
            print("❌ No se pudo generar el reporte")
            print("💡 Verifica que los archivos necesarios estén cargados")

def prediccion_ml(button):
    """Predicción de ventas usando ML"""
    with output_ml:
        clear_output()

        # Verificamos que exista el CSV de ventas
        if "ventas.csv" not in dataframes:
            print("⚠️ No se encontró el archivo 'ventas.csv'")
            return

        df = dataframes["ventas.csv"]
        df_num = df.select_dtypes(include='number')

        if len(df_num.columns) < 2:
            print("❌ Se necesitan al menos 2 columnas numéricas para predecir")
            return

        print("🔮 PREDICCIÓN DE VENTAS")
        print("="*50)

        # Limpiamos datos nulos
        df_clean = df_num.dropna()
        if len(df_clean) < 10:
            print("❌ Datos insuficientes (mínimo 10 registros)")
            return

        # Última columna como variable a predecir (cantidad)
        X = df_clean.iloc[:, :-1]  # Predictores (ej: precio, id_producto)
        y = df_clean.iloc[:, -1]   # Cantidad vendida

        # Entrenamiento
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        modelo = LinearRegression()
        modelo.fit(X_train, y_train)
        y_pred = modelo.predict(X_test)

        # Mostrar resultados de forma simple
        df_resultados = pd.DataFrame({
            'Ventas Reales': y_test.values,
            'Predicción de Ventas': y_pred
        })

        print("📋 Predicciones de ventas (muestra):")
        display(df_resultados.head(10))

        # Explicación para el cliente
        print("\n💡 Qué significa esto para su negocio:")
        print("- El modelo analiza ventas pasadas y las variables asociadas (producto, precio, sucursal).")
        print("- Con esto, estima cuántas unidades se venderían en condiciones similares.")
        print("- Los puntos en el gráfico muestran ventas reales vs. estimadas. La línea azul indica un ajuste ideal.")

        # Gráfico simple
        fig, ax = plt.subplots(figsize=(8,5))
        ax.scatter(y_test, y_pred, color='red', alpha=0.7)
        ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'b--')
        ax.set_xlabel("Ventas Reales")
        ax.set_ylabel("Predicción de Ventas")
        ax.set_title("Predicción de Ventas vs Real")
        plt.tight_layout()
        plt.show()


def exportar_excel(button):
    """Exporta análisis completo a Excel"""
    with output_exportar:
        clear_output()

        nombre = combo_analisis.value

        if not nombre or nombre not in dataframes:
            print("⚠️ Selecciona un archivo")
            return

        df = dataframes[nombre]

        print("💾 EXPORTANDO A EXCEL")
        print("="*70)

        excel_path = nombre.replace('.csv', '_analisis_completo.xlsx')

        try:
            with pd.ExcelWriter(excel_path, engine='xlsxwriter') as writer:
                # Datos originales
                df.to_excel(writer, sheet_name='Datos', index=False)
                print("✅ Hoja 'Datos' creada")

                # Estadísticas
                df_num = df.select_dtypes(include='number')
                if not df_num.empty:
                    df_num.describe().to_excel(writer, sheet_name='Estadísticas')
                    print("✅ Hoja 'Estadísticas' creada")

                # Correlación
                if len(df_num.columns) >= 2:
                    df_num.corr().to_excel(writer, sheet_name='Correlación')
                    print("✅ Hoja 'Correlación' creada")

                # Historial
                if historial_cambios:
                    pd.DataFrame(historial_cambios).to_excel(writer, sheet_name='Historial', index=False)
                    print("✅ Hoja 'Historial' creada")

            print(f"\n✅ Archivo exportado: {excel_path}")
            print("📥 Descárgalo del panel lateral de archivos")

        except Exception as e:
            print(f"❌ Error al exportar: {e}")

def descargar_csv_json(button):
    """Descarga archivo en CSV o JSON"""
    with output_descargar:
        clear_output()

        nombre = combo_analisis.value

        if not nombre or nombre not in dataframes:
            print("⚠️ Selecciona un archivo")
            return

        df = dataframes[nombre]

        print("📥 DESCARGAR ARCHIVO")
        print("="*70)
        print("Formatos disponibles:")
        print("1. CSV")
        print("2. JSON")

        formato = input("\nSelecciona formato (1/2): ").strip()

        if formato == "1":
            archivo = nombre.replace('.csv', '_modificado.csv')
            df.to_csv(archivo, index=False)
            print(f"✅ Guardado: {archivo}")
        elif formato == "2":
            archivo = nombre.replace('.csv', '_modificado.json')
            df.to_json(archivo, orient='records', indent=2)
            print(f"✅ Guardado: {archivo}")
        else:
            print("❌ Opción inválida")
            return

        print("📥 Descárgalo del panel lateral de archivos")

def ver_historial(button):
    """Muestra historial de cambios"""
    with output_historial:
        clear_output()

        print("📜 HISTORIAL DE CAMBIOS")
        print("="*70)

        if not historial_cambios:
            print("ℹ️ No hay cambios registrados")
            return

        df_historial = pd.DataFrame(historial_cambios)
        display(df_historial)

        print(f"\n📊 Total de operaciones: {len(historial_cambios)}")

# ============================================================================
# WIDGETS
# ============================================================================

# CARGA
uploader = widgets.FileUpload(
    accept='.csv',
    multiple=True,
    description='📤 Upload',
    button_style='info'
)

lista_archivos = widgets.HTML(
    value='<p style="color: gray;">📂 No hay archivos cargados</p>'
)

output_carga = widgets.Output()

# VISUALIZACIÓN
combo_csv = widgets.Dropdown(
    description='Archivo:',
    options=[],
    layout=widgets.Layout(width='80%')
)

btn_ver_datos = widgets.Button(
    description="👁️ Ver Datos Completos",
    button_style='primary',
    layout=widgets.Layout(width='50%')
)

output_visualizar = widgets.Output()

# MODIFICAR
combo_modificar = widgets.Dropdown(
    description='Archivo:',
    options=[],
    layout=widgets.Layout(width='80%')
)

btn_anadir = widgets.Button(
    description="➕ Añadir Fila",
    button_style='success'
)

btn_modificar_celda = widgets.Button(
    description="✏️ Modificar Celda",
    button_style='warning'
)

output_modificar = widgets.Output()

# ELIMINAR
combo_eliminar = widgets.Dropdown(
    description='Archivo:',
    options=[],
    layout=widgets.Layout(width='80%')
)

btn_eliminar = widgets.Button(
    description="🗑️ Eliminar Fila",
    button_style='danger',
    layout=widgets.Layout(width='50%')
)

output_eliminar = widgets.Output()

# MERGE
combo_merge = widgets.Dropdown(
    description='Base:',
    options=[],
    layout=widgets.Layout(width='80%')
)

btn_merge = widgets.Button(
    description="🔗 Unificar Tablas",
    button_style='info',
    layout=widgets.Layout(width='50%')
)

output_merge = widgets.Output()

# REPORTES
combo_reportes = widgets.Dropdown(
    description='Reporte:',
    options=[
        '🏆 Ranking de Clientes',
        '💰 Ticket Promedio',
        '📅 Ventas por Mes',
        '🎯 Top Productos'
    ],
    layout=widgets.Layout(width='80%')
)

btn_reporte = widgets.Button(
    description="📊 Ejecutar Reporte",
    button_style='success',
    layout=widgets.Layout(width='50%')
)

output_reportes = widgets.Output()

# MACHINE LEARNING
combo_analisis = widgets.Dropdown(
    description='Dataset:',
    options=[],
    layout=widgets.Layout(width='80%')
)

btn_ml = widgets.Button(
    description="🔮 Predicción ML",
    button_style='primary',
    layout=widgets.Layout(width='50%')
)

output_ml = widgets.Output()

# EXPORTAR
btn_exportar_excel = widgets.Button(
    description="💾 Exportar a Excel",
    button_style='success'
)

btn_descargar = widgets.Button(
    description="📥 Descargar CSV/JSON",
    button_style='info'
)

output_exportar = widgets.Output()
output_descargar = widgets.Output()

# HISTORIAL
btn_historial = widgets.Button(
    description="📜 Ver Historial",
    button_style='warning',
    layout=widgets.Layout(width='50%')
)

output_historial = widgets.Output()

# ============================================================================
# EVENTOS
# ============================================================================

uploader.observe(on_upload_change, names='value')
btn_ver_datos.on_click(ver_datos_completos)
btn_anadir.on_click(anadir_fila)
btn_modificar_celda.on_click(modificar_celda)
btn_eliminar.on_click(eliminar_fila)
btn_merge.on_click(unificar_tablas)
btn_reporte.on_click(ejecutar_reporte)
btn_ml.on_click(prediccion_ml)
btn_exportar_excel.on_click(exportar_excel)
btn_descargar.on_click(descargar_csv_json)
btn_historial.on_click(ver_historial)

# ============================================================================
# INFORME HTML
# ============================================================================

informe_html = """
<div style="font-family: Arial, sans-serif; line-height: 1.6; padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); border-radius: 15px; color: white;">
    <h1 style="text-align: center; font-size: 2.5em; margin-bottom: 20px;">
        EXPLICACION
    </h1>
    <p style="text-align: center; font-size: 1.2em; margin-bottom: 30px;">
        Sistema Completo de Gestión, Análisis y Predicción de Datos
    </p>

    <div style="background-color: rgba(255,255,255,0.1); padding: 20px; border-radius: 10px; margin: 20px 0;">
        <h3 style="color: #ffd700;">🎯 Funcionalidades</h3>

        <div style="margin: 15px 0;">
            <h4>📁 Gestión de Datos</h4>
            <h5>Usa las pestañas superiores para navegar</h5>
            <h5>Comienza subiendo archivos en la pestaña '📤 Cargar'</h5>
            <ul>
                <li>Carga múltiple de archivos CSV</li>
                <li>Visualización completa sin omitir registros</li>
                <li>Añadir filas con IDs autoincrementales</li>
                <li>Modificar celdas específicas</li>
                <li>Eliminar filas por índice</li>
            </ul>
        </div>

        <div style="margin: 15px 0;">
            <h4>🔧 Operaciones Avanzadas</h4>
            <ul>
                <li>Unificar tablas con MERGE (LEFT, RIGHT, INNER, OUTER)</li>
                <li>Selección de columna común para unión</li>
                <li>Guardado de resultados unificados</li>
            </ul>
        </div>

        <div style="margin: 15px 0;">
            <h4>📊 Reportes y Análisis</h4>
            <ul>
                <li>Ranking de clientes por facturación</li>
                <li>Ticket promedio por cliente</li>
                <li>Ventas mensuales</li>
                <li>Top productos más vendidos</li>
                <li>Gráficos automáticos</li>
            </ul>
        </div>

        <div style="margin: 15px 0;">
            <h4>🔮 Machine Learning</h4>
            <ul>
                <li>Regresión Lineal automática</li>
                <li>División train/test 80/20</li>
                <li>Métricas: R² Score y RMSE</li>
                <li>Gráficos de predicción</li>
            </ul>
        </div>

        <div style="margin: 15px 0;">
            <h4>💾 Exportación</h4>
            <ul>
                <li>Excel con múltiples hojas</li>
                <li>CSV modificado</li>
                <li>JSON para APIs</li>
                <li>Historial de cambios</li>
            </ul>
        </div>
    </div>

    <div style="background-color: rgba(255,255,255,0.15); padding: 15px; border-radius: 10px; margin: 20px 0;">
        <h3 style="color: #ffd700;">🚀 Guía de Uso Rápido</h3>
        <ol style="font-size: 1.1em; line-height: 2;">
            <li>📤 <strong>Sube tus archivos CSV</strong> en la pestaña "Cargar Archivos"</li>
            <li>👁️ <strong>Visualiza</strong> tus datos completos sin omisiones</li>
            <li>✏️ <strong>Modifica</strong> filas, columnas o añade nuevos registros</li>
            <li>🔗 <strong>Unifica</strong> múltiples tablas si es necesario</li>
            <li>📊 <strong>Genera reportes</strong> automáticos predefinidos</li>
            <li>🔮 <strong>Predice</strong> valores con Machine Learning</li>
            <li>💾 <strong>Exporta</strong> en Excel, CSV o JSON</li>
        </ol>
    </div>

    <div style="text-align: center; margin-top: 30px; padding: 15px; background-color: rgba(255,255,255,0.1); border-radius: 10px;">
        <p style="font-size: 1.2em; margin: 0;">
            ✨ <strong>¡Listo para comenzar!</strong> ✨<br>
            <small>Ve a la pestaña "Cargar Archivos" para empezar</small>
        </p>
    </div>
</div>
"""

# ============================================================================
# INTERFAZ PRINCIPAL CON TABS
# ============================================================================

informe_widget = widgets.HTML(value=informe_html)

tab_contenido = widgets.Tab()
tab_contenido.children = [
    # TAB 1: INFORME
    widgets.VBox([informe_widget]),

    # TAB 2: CARGAR ARCHIVOS
    widgets.VBox([
        widgets.HTML('<h2 style="color: #007bff;">📤 Cargar Archivos CSV</h2>'),
        widgets.HTML('<p>Selecciona uno o múltiples archivos CSV. Se acumularán sin reemplazar.</p>'),
        uploader,
        lista_archivos,
        output_carga
    ]),

    # TAB 3: VISUALIZAR
    widgets.VBox([
        widgets.HTML('<h2 style="color: #007bff;">👁️ Visualizar Datos Completos</h2>'),
        widgets.HTML('<p>Muestra TODOS los registros sin omitir ninguno.</p>'),
        combo_csv,
        btn_ver_datos,
        output_visualizar
    ]),

    # TAB 4: MODIFICAR
    widgets.VBox([
        widgets.HTML('<h2 style="color: #007bff;">✏️ Modificar Datos</h2>'),
        combo_modificar,
        widgets.HBox([btn_anadir, btn_modificar_celda]),
        widgets.HTML('<br><p><strong>Añadir Fila:</strong> Los IDs se generan automáticamente</p>'),
        widgets.HTML('<p><strong>Modificar Celda:</strong> Cambia un valor específico</p>'),
        output_modificar
    ]),

    # TAB 5: ELIMINAR
    widgets.VBox([
        widgets.HTML('<h2 style="color: #007bff;">🗑️ Eliminar Datos</h2>'),
        combo_eliminar,
        btn_eliminar,
        widgets.HTML('<p>Elimina una fila por su índice (0 = primera fila)</p>'),
        output_eliminar
    ]),

    # TAB 6: UNIFICAR
    widgets.VBox([
        widgets.HTML('<h2 style="color: #007bff;">🔗 Unificar Tablas (MERGE)</h2>'),
        widgets.HTML('<p>Une múltiples archivos usando una columna común</p>'),
        combo_merge,
        btn_merge,
        output_merge
    ]),

    # TAB 7: REPORTES
    widgets.VBox([
        widgets.HTML('<h2 style="color: #007bff;">📊 Reportes Automáticos</h2>'),
        widgets.HTML('<p>Genera análisis predefinidos con gráficos</p>'),
        combo_reportes,
        btn_reporte,
        output_reportes
    ]),

    # TAB 8: MACHINE LEARNING
    widgets.VBox([
        widgets.HTML('<h2 style="color: #007bff;">🔮 Predicción con ML</h2>'),
        widgets.HTML('<p>Regresión Lineal automática con visualización</p>'),
        btn_ml,
        output_ml
    ]),

    # TAB 9: EXPORTAR
    widgets.VBox([
        widgets.HTML('<h2 style="color: #007bff;">💾 Exportar Datos</h2>'),
        combo_analisis,
        widgets.HBox([btn_exportar_excel, btn_descargar]),
        widgets.HTML('<br><p><strong>Excel:</strong> Análisis completo con múltiples hojas</p>'),
        widgets.HTML('<p><strong>CSV/JSON:</strong> Archivo modificado en el formato elegido</p>'),
        output_exportar,
        output_descargar
    ]),

    # TAB 10: HISTORIAL
    widgets.VBox([
        widgets.HTML('<h2 style="color: #007bff;">📜 Historial de Cambios</h2>'),
        widgets.HTML('<p>Registro de todas las operaciones realizadas</p>'),
        btn_historial,
        output_historial
    ])
]

# TÍTULOS DE LAS PESTAÑAS
tab_contenido.set_title(0, '📄 Inicio')
tab_contenido.set_title(1, '📤 Cargar')
tab_contenido.set_title(2, '👁️ Ver')
tab_contenido.set_title(3, '✏️ Modificar')
tab_contenido.set_title(4, '🗑️ Eliminar')
tab_contenido.set_title(5, '🔗 Unificar')
tab_contenido.set_title(6, '📊 Reportes')
tab_contenido.set_title(7, '🔮 ML')
tab_contenido.set_title(8, '💾 Exportar')
tab_contenido.set_title(9, '📜 Historial')

# ============================================================================
# MOSTRAR INTERFAZ
# ============================================================================

# Header
header = widgets.HTML("""
<div style="background: linear-gradient(135deg, #1e3c72 0%, #2a5298 100%);
            padding: 30px;
            border-radius: 15px;
            text-align: center;
            color: white;
            margin-bottom: 20px;
            box-shadow: 0 5px 15px rgba(0,0,0,0.3);">
    <h1 style="margin: 0; font-size: 3em; text-shadow: 2px 2px 4px rgba(0,0,0,0.3);">
        📊 DATA MANAGER DASHBOARD
    </h1>
    <p style="margin: 5px 0 0 0; font-size: 0.9em; opacity: 0.8; color:red;">
        Google Colab Edition 2025
    </p>
</div>
""")

# Footer
footer = widgets.HTML("""
<div style="background-color: #f8f9fa;
            padding: 20px;
            border-radius: 10px;
            text-align: center;
            margin-top: 20px;
            border-top: 3px solid #007bff;">
    <p style="margin: 5px 0; color: #6c757d;">
        <strong>Desarrollado para Procesamiento de Aprendizaje Automático</strong>
    </p>
    <p style="margin: 5px 0; color: #007bff; font-size: 0.85em;">
        ⭐ Echo por Maximiliano Facundo Stella Zapata, 2do año de Ciencia de datos e Inteligencia Artificial ⭐
    </p>
</div>
""")

# DISPLAY FINAL
display(header)
display(tab_contenido)
display(footer)

HTML(value='\n<div style="background: linear-gradient(135deg, #1e3c72 0%, #2a5298 100%); \n            padding…

HTML(value='\n<div style="background-color: #f8f9fa; \n            padding: 20px; \n            border-radius:…


✅ SISTEMA LISTO
📊 Total de archivos cargados: 0
📝 Operaciones registradas: 0

👉 Usa las pestañas superiores para navegar
💡 Comienza subiendo archivos en la pestaña '📤 Cargar'
